# Import Library

In [45]:
import pandas as pd

import pandas as pd
import numpy as np
import tensorflow as tf

import string
import re #regex/regular expressions library for remove numbers
import ast
import nltk

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense

from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences #untuk proses padding



# Connect to Drive

In [46]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Dataset Spotify App Reviews from Kaggle

In [47]:
df = pd.read_csv('/content/drive/MyDrive/pengembangan-ML-dicoding/reviews.csv') #read csv file

telah memenuhi kriteria dataset yang dipakai yakni bersumber dari kaggle

# Data Inspection

In [48]:
df.head() #display top rows

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


In [49]:
df.tail() #display bottom rows

,Time_submitted,Review,Rating,Total_thumbsup,Reply
61589,2022-01-01 03:01:29,Even though it was communicated that lyrics fe...,1,6,NaN
61590,2022-01-01 02:13:40,"Use to be sooo good back when I had it, and wh...",1,0,NaN
61591,2022-01-01 01:02:29,This app would be good if not for it taking ov...,2,10,NaN
61592,2022-01-01 00:49:23,The app is good hard to navigate and won't jus...,2,1,NaN
61593,2022-01-01 00:19:09,Its good but sometimes it doesnt load the musi...,4,0,NaN


In [50]:
df.dtypes #display data types

Time_submitted    object
Review            object
Rating             int64
Total_thumbsup     int64
Reply             object
dtype: object

In [51]:
df.describe() #summary statistics

,Rating,Total_thumbsup
count,61594.000000,61594.000000
mean,3.155989,7.649381
std,1.673285,89.323143
min,1.000000,0.000000
25%,1.000000,0.000000
50%,3.000000,0.000000
75%,5.000000,1.000000
max,5.000000,8195.000000


In [52]:
df.info() #display index, columns and data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61594 entries, 0 to 61593
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Time_submitted  61594 non-null  object
 1   Review          61594 non-null  object
 2   Rating          61594 non-null  int64 
 3   Total_thumbsup  61594 non-null  int64 
 4   Reply           216 non-null    object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [53]:
len(df)

61594

telah memenuhi kriteria dataset yg digunakan yakni minimal 1000 sampel

tipe dataset ini termasuk kategorikal dan multiclass text classification

# Remove Columns not necessarily needed

In [54]:
df_columns_removed = df.drop(columns=['Time_submitted','Total_thumbsup','Reply'])

In [55]:
df_columns_removed.head()

,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
2,"This pop-up ""Get the best Spotify experience o...",4
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1


# Data Cleaning

In [56]:
df_columns_removed.isnull().sum() #Check for missing value

Review    0
Rating    0
dtype: int64

In [61]:
def hapus_ulasan_special(text):
    # hapus tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # hapus non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # hapus mention @
    text = re.sub(r"[@][\w_-]+","", text)
    # hapus link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # hapus incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df_columns_removed['cleaning'] = df_columns_removed['Review'].apply(hapus_ulasan_special)

#hapus number
def hapus_number(text):
    return  re.sub(r"\d+", "", text)

df_columns_removed['cleaning'] = df_columns_removed['cleaning'].apply(hapus_number)

#hapus punctuation
def hapus_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df_columns_removed['cleaning'] = df_columns_removed['cleaning'].apply(hapus_punctuation)

#hapus whitespace leading & trailing
def hapus_whitespace_LT(text):
    return text.strip()

df_columns_removed['cleaning'] = df_columns_removed['cleaning'].apply(hapus_whitespace_LT)

#hapus multiple whitespace into single whitespace
def hapus_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df_columns_removed['cleaning'] = df_columns_removed['cleaning'].apply(hapus_whitespace_multiple)

# hapus single char
def hapus_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df_columns_removed['cleaning'] = df_columns_removed['cleaning'].apply(hapus_singl_char)

#hapus pengulangan dalam kata
def replaceThreeOrMore (text):
    #Pattern to look for three or more repetitions of any character, including newlines(contoh goool -> gool)
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", text)

df_columns_removed['cleaning'] = df_columns_removed['cleaning'].apply(replaceThreeOrMore)

# hapus duplicate
#df.drop_duplicates(subset=['cleaning'], inplace=True)

df_columns_removed.loc[0:10000,['Review', 'cleaning']]

,Review,cleaning
0,"Great music service, the audio is high quality...",Great music service the audio is high quality ...
1,Please ignore previous negative rating. This a...,Please ignore previous negative rating This ap...
2,"This pop-up ""Get the best Spotify experience o...",This popup Get the best Spotify experience on ...
3,Really buggy and terrible to use as of recently,Really buggy and terrible to use as of recently
4,Dear Spotify why do I get songs that I didn't ...,Dear Spotify why do get songs that didnt put...
...,...,...
9996,"This app has become a problem, why do you alwa...",This app has become problem why do you always...
9997,"The app is good, but the lyric won't show for ...",The app is good but the lyric wont show for nu...
9998,Good App..Doesn't have ads between songs and h...,Good AppDoesnt have ads between songs and has ...
9999,Impossible to use on lock screen anymore. Trie...,Impossible to use on lock screen anymore Tried...


In [62]:
#Case Folding
df_columns_removed['case_folding'] = df_columns_removed['cleaning'].str.lower()
df_columns_removed.loc[0:10000,['cleaning', 'case_folding']]

,cleaning,case_folding
0,Great music service the audio is high quality ...,great music service the audio is high quality ...
1,Please ignore previous negative rating This ap...,please ignore previous negative rating this ap...
2,This popup Get the best Spotify experience on ...,this popup get the best spotify experience on ...
3,Really buggy and terrible to use as of recently,really buggy and terrible to use as of recently
4,Dear Spotify why do get songs that didnt put...,dear spotify why do get songs that didnt put...
...,...,...
9996,This app has become problem why do you always...,this app has become problem why do you always...
9997,The app is good but the lyric wont show for nu...,the app is good but the lyric wont show for nu...
9998,Good AppDoesnt have ads between songs and has ...,good appdoesnt have ads between songs and has ...
9999,Impossible to use on lock screen anymore Tried...,impossible to use on lock screen anymore tried...


In [69]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [82]:
#Stopwords

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#get stopwords english
stop_words = stopwords.words('english')

# convert list to dictionary
stop_words = set(stopwords.words('english'))

example_sent = """This is a sample sentence, showing off the stop words filtration."""
word_tokens = word_tokenize(example_sent)


#remove stopword pada list token
def filtered_sentence(word_tokens):
    #kata = [word for word in words if word not in list_stopwords]
    return [w for w in word_tokens if not w.lower() in stop_words]

df_columns_removed['stopwords'] = df_columns_removed['case_folding'].apply(filtered_sentence)
df_columns_removed.loc[0:10000,['case_folding', 'stopwords']]

,case_folding,stopwords
0,great music service the audio is high quality ...,"[g, r, e, , u, c, , e, r, v, c, e, , h, e, ..."
1,please ignore previous negative rating this ap...,"[p, l, e, e, , g, n, r, e, , p, r, e, v, u, ..."
2,this popup get the best spotify experience on ...,"[h, , p, p, u, p, , g, e, , h, e, , b, e, ..."
3,really buggy and terrible to use as of recently,"[r, e, l, l, , b, u, g, g, , n, , e, r, r, ..."
4,dear spotify why do get songs that didnt put...,"[e, r, , p, f, , w, h, , , , g, e, , n, ..."
...,...,...
9996,this app has become problem why do you always...,"[h, , p, p, , h, , b, e, c, e, , , p, r, ..."
9997,the app is good but the lyric wont show for nu...,"[h, e, , p, p, , , g, , b, u, , h, e, , ..."
9998,good appdoesnt have ads between songs and has ...,"[g, , p, p, e, n, , h, v, e, , , b, e, w, ..."
9999,impossible to use on lock screen anymore tried...,"[p, b, l, e, , , u, e, , n, , l, c, k, , ..."


# Lakukan ONE HOT ENDOING KARENA INI TIPE DATA KATEGORIKAL

In [85]:
category = pd.get_dummies(df_columns_removed.Rating)
df_cleaned = pd.concat([df_columns_removed, category], axis=1)
df_cleaned = df_cleaned.drop(columns='Rating')
df_cleaned

,Review,cleaning,case_folding,stopwords,1,2,3,4,5
0,"Great music service, the audio is high quality...",Great music service the audio is high quality ...,great music service the audio is high quality ...,"[g, r, e, , u, c, , e, r, v, c, e, , h, e, ...",0,0,0,0,1
1,Please ignore previous negative rating. This a...,Please ignore previous negative rating This ap...,please ignore previous negative rating this ap...,"[p, l, e, e, , g, n, r, e, , p, r, e, v, u, ...",0,0,0,0,1
2,"This pop-up ""Get the best Spotify experience o...",This popup Get the best Spotify experience on ...,this popup get the best spotify experience on ...,"[h, , p, p, u, p, , g, e, , h, e, , b, e, ...",0,0,0,1,0
3,Really buggy and terrible to use as of recently,Really buggy and terrible to use as of recently,really buggy and terrible to use as of recently,"[r, e, l, l, , b, u, g, g, , n, , e, r, r, ...",1,0,0,0,0
4,Dear Spotify why do I get songs that I didn't ...,Dear Spotify why do get songs that didnt put...,dear spotify why do get songs that didnt put...,"[e, r, , p, f, , w, h, , , , g, e, , n, ...",1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
61589,Even though it was communicated that lyrics fe...,Even though it was communicated that lyrics fe...,even though it was communicated that lyrics fe...,"[e, v, e, n, , h, u, g, h, , , w, , c, u, ...",1,0,0,0,0
61590,"Use to be sooo good back when I had it, and wh...",Use to be soo good back when had it and when ...,use to be soo good back when had it and when ...,"[u, e, , , b, e, , , g, , b, c, k, , w, ...",1,0,0,0,0
61591,This app would be good if not for it taking ov...,This app would be good if not for it taking ov...,this app would be good if not for it taking ov...,"[h, , p, p, , w, u, l, , b, e, , g, , f, ...",0,1,0,0,0
61592,The app is good hard to navigate and won't jus...,The app is good hard to navigate and wont just...,the app is good hard to navigate and wont just...,"[h, e, , p, p, , , g, , h, r, , , n, v, ...",0,1,0,0,0


# Cleaned Dataset with Necessary Columns Needed

In [86]:
df2 = df_cleaned[['stopwords', 1, 2, 3, 4, 5]]

In [87]:
df2

,stopwords,1,2,3,4,5
0,"[g, r, e, , u, c, , e, r, v, c, e, , h, e, ...",0,0,0,0,1
1,"[p, l, e, e, , g, n, r, e, , p, r, e, v, u, ...",0,0,0,0,1
2,"[h, , p, p, u, p, , g, e, , h, e, , b, e, ...",0,0,0,1,0
3,"[r, e, l, l, , b, u, g, g, , n, , e, r, r, ...",1,0,0,0,0
4,"[e, r, , p, f, , w, h, , , , g, e, , n, ...",1,0,0,0,0
...,...,...,...,...,...,...
61589,"[e, v, e, n, , h, u, g, h, , , w, , c, u, ...",1,0,0,0,0
61590,"[u, e, , , b, e, , , g, , b, c, k, , w, ...",1,0,0,0,0
61591,"[h, , p, p, , w, u, l, , b, e, , g, , f, ...",0,1,0,0,0
61592,"[h, e, , p, p, , , g, , h, r, , , n, v, ...",0,1,0,0,0


# Convert to Array

In [88]:
ulasan = df2['stopwords'].values
label = df2[[1, 2, 3, 4, 5]].values

# Split Data into Train (80%) and Test (20%)

In [89]:
from sklearn.model_selection import train_test_split
ulasan_latih, ulasan_test, label_latih, label_test = train_test_split(ulasan, label, test_size=0.2)

# Tokenizing, Sequencing converted

In [90]:
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(ulasan_latih)
tokenizer.fit_on_texts(ulasan_test)

sekuens_latih = tokenizer.texts_to_sequences(ulasan_latih)
sekuens_test = tokenizer.texts_to_sequences(ulasan_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

telah memenuhi kriteria: menggunakan fungsi tokenizer

# Build Model using layer embedding, LSTM

In [91]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

telah memenuhi kriteria: harus menggunakan LSTM dalam arsitektur model

telah memenuhi kriteria: Harus menggunakan model sequential

telah memenuhi kriteria: Harus menggunakan Embedding




# Callback Implementation

In [94]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

# Train Model

In [ ]:
num_epochs = 50
history = model.fit(padded_latih, label_latih, epochs=num_epochs, callbacks=[callbacks],
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/50


telah memenuhi kriteria: Validation set sebesar 20% dari total dataset

telah memenuhi kriteria: Akurasi dari model minimal 75% pada train set dan validation set.